##Scheduling College Bowl Games

I recently read an article that mentioned that some schools, who had a 6-6 record were irritated at the inclusion of the University of Nebraska, a team with a losing 5-7 record in the Faster Farms Bowl.  Though the Huskers ended up beating UCLA, the drama made me wonder how the scheduling is done and whether I could prototype something to automate scheduling.

First, my infrastructure consists of Anaconda Python, Coopr.pyomo and Gurobi.  Interestingly, those with an academic email can get a free version of gurobi. Not a bad software setup considering it is all open source or free! It's worth noting that I ran into some version issues with pyomo, which I solved by reverting back to Python 2.7.  Otherwise, everything was pretty straight forward to install.

Back to the model. I modeled this as a bipartite matching problem where we have two sets, Bowl Games and Schools.  In fact, this problem is almost the exact same as the work I did in 2011, where we scheduled University Suppliers to campus Supplier Shows.  In this case, the event is a bowl game and the suppliers are Universities.  I then use a binary decicion to indicate whether school $i$ is invited to bowl game $j$.  With some constraints to make a reasonable schedule and with a thoughtless (i.e. I didnt think very much about this) objective function, I optimize and generate the schedule. 

The mathmatical model for this model is as follows:

###Sets
$I$ = set of schools <br>
$J$ = set of bowl games

###Parameters
$t_{ij}$ = travel miles between school $i$ and bowl game $j$,  $\ {\forall }i\ {\in}\ I, \ {\forall}j\ {\in}\ J$<br>
$a_i$= Academic Progress Statistic for school $i$, $\ {\forall}i\ {\in}\ I$<br>
$s_i$= Strength of Schedule for school $i$, $\ {\forall}i\ {\in}\ I$<br>

###Variables
$x = \begin{cases}
  0, & \text{if  School $i$ is invited to bowl game $j$}, \\
  1, & \text{otherwise}.
\end{cases}$

###Objective
$Max \sum\limits_{i\ {\in}\ I} \sum\limits_{j\ {\in}\ J}x_{ij}*a_i + x_{ij}*s_i - x_{ij}*t_{ij}$

###Constraints
$\sum\limits_{i\ {\in}\ I} x_{ij} = 2,\ {\forall}i\ {\in}\ I$,  Each bowl game must have exactly 2 teams<br><br>
$\sum\limits_{j\ {\in}\ J} x_{ij} <= 1,\ {\forall}j\ {\in}\ J$, Each team is scheduled to a maximum of 1 bowl game


In [1]:
import pandas as pd
from coopr.pyomo import *
from coopr import opt
from coopr.pyomo import value
import random

Error loading 'pyutilib.component' entry points: 'type object 'PluginGlobals' has no attribute 'add_env''


###Data Tables

This is the section where we create the data for the model.  It isn't a leap to see that we could easily scrape the data off the web, rather than manually manipulating dataframes.  For examples of how to do scrape data, check out [The Berkeley Hacker Within WebScraping Tutorial](https://github.com/thehackerwithin/berkeley/blob/master/scraping/webscraping_tutorial.ipynb).  Unless the NCAA asks me to do something more legitimate, I am ok just prototyping and manually pulling some data. 

In [2]:
BowlGamesIndex = ['Longitude','Latitude']
BowlGames = pd.DataFrame({'Armed Forces Bowl': [32.7574,97.3332], 
                          'Arizona Bowl' : [32.2217, 110.9264]
                         }, index=BowlGamesIndex).transpose()

BowlGames

,Longitude,Latitude
Arizona Bowl,32.2217,110.9264
Armed Forces Bowl,32.7574,97.3332


In [3]:
SchoolsIndex = ['AcademicRank', 'StrengthOfSchedule', 'Longitude', 'Latitude', 'Wins']
Schools = pd.DataFrame({'California': [941, 7.8,37.8700,122.2590,8],
                        'Nevada': [943, 6.7,47.6550, 122.3080,6],
                        'UCLA': [975, 5.7,34.0722, 118.4441,9],
                        'Arizona State':[949,5.5,33.4172, 111.9365,6],
                        'Colorado': [957,-0.4,40.0067, 105.2672,4],
                        'Stanford': [987,7.8,37.4300, 122.1700,11]
                        }, index=SchoolsIndex).transpose()

Schools.AcademicRank = Schools.AcademicRank.astype(float)
Schools.StrengthOfSchedule = Schools.StrengthOfSchedule.astype(float)
Schools

,AcademicRank,StrengthOfSchedule,Longitude,Latitude,Wins
Arizona State,949,5.5,33.4172,111.9365,6
California,941,7.8,37.8700,122.2590,8
Colorado,957,-0.4,40.0067,105.2672,4
Nevada,943,6.7,47.6550,122.3080,6
Stanford,987,7.8,37.4300,122.1700,11
UCLA,975,5.7,34.0722,118.4441,9


###PreProcess the inputs
There are some results that we wouldn't want to see.  Rather than constraining the model, we can just as easily modify the inputs to preclude assignment.  Doing it in preprocessing is 1) easier because I am better at Pandas than Pyomo and 2) it reduces the size and complexity of the model.  

In [4]:
#Print the schools who will be excluded
print(Schools[Schools['Wins'] < 6])

#Remove those schools who dont have 6 or more wins
Schools = Schools[Schools['Wins'] >= 6]

          AcademicRank  StrengthOfSchedule  Longitude  Latitude  Wins
Colorado           957                -0.4    40.0067  105.2672     4


###Creating the distance parameter
My friend Ramsey tells me that Universities have to pay for their own trips to their bowl game.  I would then assume that the further you travel the further you are from your fan base and thus would have lower interest in attending the game.  For example, imagine if San Jose State University had to travel to Anchorage Alaska for a bowl game.  I cant imagine there would be the interest to justify the trip. 

Based on this assumption, I created a parameter that measures the approximate distance each school is from each game. While I assume there are several ways to calculate distance, I used the first code I happened upon from StackOverflow.

Ultimately, this metric is all about demand for the game in specific markets.  Fréchette et al. 2007 use the Neilson TV ratings, which is better. It doesnt seem to difficult to imagine using web traffic (e.g. Google Searches) or Neilson data or any number of other sources to gather insight into where college teams have strong fan bases.  Thus allowing the creation of a schedule which meets the highest level of demand and fills up stadiums.   

In [5]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

a = {}
#Loop through all the bowl games and teams and calculate the travel distance in KM 
for i,j in [(i,j) for i in Schools.index for j in BowlGames.index]:
    #run the haversine function
    distance = haversine(Schools.Longitude[i], Schools.Latitude[i], 
                         BowlGames.Longitude[j], BowlGames.Latitude[j])
   
    a[i,j] =  distance

a

{('Arizona State', 'Arizona Bowl'): 122.28850561362388,
 ('Arizona State', 'Armed Forces Bowl'): 1622.8732094381455,
 ('California', 'Arizona Bowl'): 1288.982469028836,
 ('California', 'Armed Forces Bowl'): 2773.9743482000076,
 ('Nevada', 'Arizona Bowl'): 1470.411706196987,
 ('Nevada', 'Armed Forces Bowl'): 2809.7074808120256,
 ('Stanford', 'Arizona Bowl'): 1274.8307187822527,
 ('Stanford', 'Armed Forces Bowl'): 2763.413967317055,
 ('UCLA', 'Arizona Bowl'): 839.7117970568663,
 ('UCLA', 'Armed Forces Bowl'): 2346.2340941597877}

###Normalize the Paramaters
When I first ran the model without normalized paramaters, I noticed the objective function was negative.  This gave me some intuition that the scale of the parameters was too significantly different to simply multiply them together.  

In [6]:
#Normalize the dictionary of distances
factor=1.0/sum(a.itervalues())
for i,j in a:
    a[i,j] = a[i,j]*factor
    
a

{('Arizona State', 'Arizona Bowl'): 0.007063625247626339,
 ('Arizona State', 'Armed Forces Bowl'): 0.09374035702180145,
 ('California', 'Arizona Bowl'): 0.07445416939468649,
 ('California', 'Armed Forces Bowl'): 0.16023022886650137,
 ('Nevada', 'Arizona Bowl'): 0.0849338799274789,
 ('Nevada', 'Armed Forces Bowl'): 0.16229424507496265,
 ('Stanford', 'Arizona Bowl'): 0.07363673639198298,
 ('Stanford', 'Armed Forces Bowl'): 0.15962024044072845,
 ('UCLA', 'Arizona Bowl'): 0.04850340938142727,
 ('UCLA', 'Armed Forces Bowl'): 0.1355231082528041}

###The Pyomo Formulation
Now that I have all the data I need, it is time to begin coding the model.  In my experimentation with pyomo and jupyter notebook, I find it very convinient to use concrete models loaded with the data tables/lists/dictionaries I created in the cells above. This simple convention allows me to prototype with small data while testing the results and then to scale up to full datasets by simply changing some input code.



In [8]:
#Create the model object
Schedule = ConcreteModel()

#Identify our sets of entities
Schedule.Schools = Set(initialize=Schools.index.values)
Schedule.BowlGames = Set(initialize=BowlGames.index.values)

#Identify some parameters
Schedule.TravelKMs = Param(Schedule.Schools,Schedule.BowlGames,initialize=a)
Schedule.Academics = Param(Schedule.Schools, initialize=Schools.AcademicRank.to_dict())
Schedule.StrengthOfSchedule = Param(Schedule.Schools, initialize=Schools.StrengthOfSchedule.to_dict())
Schedule.Wins = Param(Schedule.Schools, initialize=Schools.Wins.to_dict())

#Set the Binary Decision Variable, which represents whether a school will be invited to a specific bowl Game.
Schedule.X = Var(Schedule.Schools, Schedule.BowlGames, within=Binary)

#Set the objective function
def obj_rule(Schedule):
    return sum(Schedule.X[i,j] * Schedule.Academics[i] + 
               Schedule.X[i,j] * Schedule.StrengthOfSchedule[i] -
               Schedule.X[i,j] * Schedule.TravelKMs[i,j]
               for i in Schedule.Schools for j in Schedule.BowlGames)

Schedule.obj = Objective(rule=obj_rule, sense=maximize)

#Set a constraint that each game gets exactly two schools
def TwoTeams_rule(Schedule, j):
    return sum(Schedule.X[i,j] for i in Schedule.Schools) == 2
Schedule.TwoTeams = Constraint(Schedule.BowlGames, rule=TwoTeams_rule)

#Set a constraint that each team gets at most 1 bowl game
def OneGame_rule(Schedule, i):
    return sum(Schedule.X[i,j] for j in Schedule.BowlGames) <= 1
Schedule.OneGame = Constraint(Schedule.Schools, rule=OneGame_rule)

#Create the model instance
instance = Schedule.create()

# create a connection to some solver
gurobi = opt.SolverFactory('gurobi')

#Solve the model 
results = gurobi.solve(instance)

instance.load(results);

###Bowl Game Matches

In [9]:
#loop over all the X variables and print out the X's with value == 1
for index in instance.X:
    if instance.X[index].value == 1:
        print instance.X[index], instance.X[index].value

X[Nevada,Armed Forces Bowl] 1.0
X[UCLA,Arizona Bowl] 1.0
X[Arizona State,Arizona Bowl] 1.0
X[Stanford,Armed Forces Bowl] 1.0


###Solver Results

In [10]:
for i in results.Problem:
    print i


Name: 
Lower bound: 3879.32251848
Upper bound: 3879.32251848
Number of objectives: 1
Number of constraints: 8
Number of variables: 11
Number of binary variables: 10
Number of integer variables: 10
Number of continuous variables: 1
Number of nonzeros: 21
Sense: maximize



###Parameters Used

In [11]:
for p in instance.active_components(Param):
    print "Parameter",p
    parmobject = getattr(instance, p)
    for index in parmobject:
        print "   ",index, parmobject[index]

Parameter TravelKMs
    ('Nevada', 'Arizona Bowl') 0.0849338799275
    ('Arizona State', 'Armed Forces Bowl') 0.0937403570218
    ('Stanford', 'Arizona Bowl') 0.073636736392
    ('California', 'Arizona Bowl') 0.0744541693947
    ('UCLA', 'Armed Forces Bowl') 0.135523108253
    ('California', 'Armed Forces Bowl') 0.160230228867
    ('Arizona State', 'Arizona Bowl') 0.00706362524763
    ('Stanford', 'Armed Forces Bowl') 0.159620240441
    ('Nevada', 'Armed Forces Bowl') 0.162294245075
    ('UCLA', 'Arizona Bowl') 0.0485034093814
Parameter Academics
    Nevada 943.0
    California 941.0
    UCLA 975.0
    Arizona State 949.0
    Stanford 987.0
Parameter StrengthOfSchedule
    Nevada 6.7
    California 7.8
    UCLA 5.7
    Arizona State 5.5
    Stanford 7.8
Parameter Wins
    Nevada 6.0
    California 8.0
    UCLA 9.0
    Arizona State 6.0
    Stanford 11.0


###Improvements
The goal of this model was to rapidly prototype a method to automate the scheduling of NCAA Bowl Games.  Now that I have a working analytic pipeline, a couple of key points could be improved.  Namely:

- Use web scraping rather than manual data collection.
- Put some thought into the objective function.  There is opportunity to match supply and demand by figuring out where teams have major markets and then setting that as part of the objective function.  
- Rethink the normalization of parameters.  Im not convinced I am not fully convinced that I am doing the right thing by normalizing.  For example, constraining a normalized parameter seems like a challenge.
- Pull out all of the numbers from the model and make them variables.  For that matter, make the user interface cleaner.

###Literature Review

Clark, Andrew G., Susan Cholette, and Ozgur Ozluk. "UCSF increases consumer value through optimal vendor-show scheduling." Interfaces 41.4 (2011): 327-337.

Fréchette, Guillaume R., Alvin E. Roth, and M. Utku Ünver. "Unraveling yields inefficient matchings: evidence from post‐season college football bowls." The RAND Journal of Economics 38.4 (2007): 967-982.

###Model Details

In [12]:
Schedule.pprint()

4 Set Declarations
    BowlGames : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Arizona Bowl', 'Armed Forces Bowl']
    Schools : Dim=0, Dimen=1, Size=5, Domain=None, Ordered=False, Bounds=None
        ['Arizona State', 'California', 'Nevada', 'Stanford', 'UCLA']
    TravelKMs_index : Dim=0, Dimen=2, Size=10, Domain=None, Ordered=False, Bounds=None
        Virtual
    X_index : Dim=0, Dimen=2, Size=10, Domain=None, Ordered=False, Bounds=None
        Virtual

4 Param Declarations
    Academics : Size=5, Index=Schools, Domain=Any, Default=None, Mutable=False
        Key           : Value
        Arizona State : 949.0
           California : 941.0
               Nevada : 943.0
             Stanford : 987.0
                 UCLA : 975.0
    StrengthOfSchedule : Size=5, Index=Schools, Domain=Any, Default=None, Mutable=False
        Key           : Value
        Arizona State :   5.5
           California :   7.8
               Nevada :   6.7
             Stanfor